In [75]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize
from scipy.interpolate import RegularGridInterpolator
import copy
import time
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.path.dirname(os.getcwd()))))
import opt_utils
import importlib
import trans_utils as T
importlib.reload(opt_utils)
importlib.reload(T)
from scipy.constants import g

In [142]:
scene_keypoints = np.array([[-0.28482594 , 0.18787911,  0.77047121],
 [-0.34903616,  0.1176751,   0.74915801],
 [-0.19674377, -0.15579415,  0.74643185],
 [-0.31491786, -0.1532551,   0.76248011],
 [-0.41689777, -0.15595002,  0.77874407]])


ee_ori = np.array([ 0.4603,  0.5229, -0.4761,  0.5366]) #xyzw

l = np.abs(scene_keypoints[4][0]) - np.abs(scene_keypoints[2][0])
apple = scene_keypoints[0]
hammer_head = apple + np.array([0,0,0.1])
hammer_handle = hammer_head - np.array([l, 0, 0])
hammer_mid = hammer_head - 0.5 * np.array([l, 0, 0])
ee_pos = hammer_handle + np.array([0,0, 0.05])
scene_keypoints[2] = hammer_handle
scene_keypoints[3] = hammer_mid
scene_keypoints[4] = hammer_head


ee_pose = np.concatenate([ee_pos, ee_ori])
print(T.quat2euler(ee_ori))
keypoints = np.concatenate([[ee_pos], scene_keypoints], axis=0)
print(keypoints)

[-0.13266497  1.54121929 -1.58026448]
[[-0.50497994  0.18787911  0.92047121]
 [-0.28482594  0.18787911  0.77047121]
 [-0.34903616  0.1176751   0.74915801]
 [-0.50497994  0.18787911  0.87047121]
 [-0.39490294  0.18787911  0.87047121]
 [-0.28482594  0.18787911  0.87047121]]


In [77]:
keypoint_movable_mask = np.zeros(keypoints.shape[0], dtype=bool)
# ee
keypoint_movable_mask[0] = True
# hammer handle
keypoint_movable_mask[3] = True
# hammer head
keypoint_movable_mask[5] = True
keypoint_movable_mask[4] = True

In [78]:
def objective(
        opt_vars, # eef pose in world frame
        og_bounds,
        keypoints_ee, # keypoints in ee frame
        keypoint_movable_mask,
        goal_constraints,
        path_constraints,
        init_pose_homo):
    
    opt_sol = opt_utils.unnormalize_vars(opt_vars, og_bounds)
    # Pose and Orientation
    opt_pose_homo = T.pose2mat([opt_sol[:3], T.euler2quat(opt_sol[3:6])])
    # Velocity
    opt_vel = opt_sol[6:]

    cost = 0.
    
    # Initial Cost
    init_pose_cost = 1.0 * opt_utils.consistency(opt_pose_homo[None], init_pose_homo[None], rot_weight=1.5)
    cost += init_pose_cost

    # subgoal constraint violation cost
    if goal_constraints is not None and len(goal_constraints) > 0:
        subgoal_constraint_cost = 0
        transformed_keypoints = opt_utils.transform_keypoints(opt_pose_homo, keypoints_ee, keypoint_movable_mask)
        subgoal_violation = []
        for constraint in goal_constraints:
            violation = constraint(transformed_keypoints[0], transformed_keypoints[1:], opt_vel)
            subgoal_violation.append(violation)
            subgoal_constraint_cost += np.clip(violation, 0, np.inf)
        subgoal_constraint_cost = 200.0*subgoal_constraint_cost
        cost += subgoal_constraint_cost
    
    # path constraint violation cost
    if path_constraints is not None and len(path_constraints) > 0:
        path_constraint_cost = 0
        transformed_keypoints = opt_utils.transform_keypoints(opt_pose_homo, keypoints_ee, keypoint_movable_mask)
        path_violation = []
        for constraint in path_constraints:
            violation = constraint(transformed_keypoints[0], transformed_keypoints[1:], opt_vel)
            path_violation.append(violation)
            path_constraint_cost += np.clip(violation, 0, np.inf)
        path_constraint_cost = 200.0*path_constraint_cost
        cost += path_constraint_cost
    return cost
    

In [79]:
ee_pose_homo = T.pose2mat([ee_pose[:3], ee_pose[3:]])
ee_pose_euler = np.concatenate([ee_pose[:3], T.quat2euler(ee_pose[3:])])
ee_pose_euler_vel = np.concatenate([ee_pose_euler, np.zeros(3)])

pos_bounds_min = [-0.45, -0.75, 0.698] #From config
pos_bounds_max = [0.2, 0.60, 1.2]
rot_bounds_min = np.array([-np.pi, -np.pi, -np.pi])  # euler angles
rot_bounds_max = np.array([np.pi, np.pi, np.pi])  # euler angles
vel_bounds_min = np.array([0.0, 0.0, 0.0])
vel_bounds_max = np.array([3.0, 0.1, 3.0])
og_bounds = [(b_min, b_max) for b_min, b_max in zip(np.concatenate([pos_bounds_min, rot_bounds_min, vel_bounds_min]), np.concatenate([pos_bounds_max, rot_bounds_max, vel_bounds_max]))]
bounds = [(-1, 1)] * len(og_bounds)

init_sol = opt_utils.normalize_vars(ee_pose_euler_vel, og_bounds)

# Center KeyPoints(world to ee frame)
centering_transform = np.linalg.inv(ee_pose_homo)
keypoints_ee = opt_utils.transform_keypoints(centering_transform, keypoints, keypoint_movable_mask)

In [131]:
def stage3_subgoal_constraint1(eef_pose, keypoints, eef_velocity):
    """
    Constraint: The hammer head (keypoint[4]) must contact the apple (keypoint[0]).
    The cost is the squared Euclidean distance between keypoint[4] and keypoint[0], minus epsilon squared.
    Constraint is satisfied if cost <= 0.
    """
    cost = np.sum((keypoints[4] - keypoints[0])**2)  
    return cost

def stage3_subgoal_constraint2(eef_pose, keypoints, eef_velocity):
    """
    Constraint: The hammer head must have sufficient velocity towards the apple at the moment of impact.
    The cost is the desired minimum velocity minus the actual velocity towards the apple.
    Constraint is satisfied if cost <= 0.
    """
    # Direction vector from hammer head to apple
    direction_vector = keypoints[0] - keypoints[4]
    direction_unit_vector = direction_vector / (np.linalg.norm(direction_vector) + 1e-8)
    # Approximate hammer head velocity as the end-effector velocity
    hammer_head_velocity = eef_velocity
    velocity_towards_apple = np.dot(hammer_head_velocity, direction_unit_vector)
    cost = 0.5 - velocity_towards_apple
    return cost

### Stage 3 path constraints


def stage3_path_constraint1(eef_pose, keypoints, eef_velocity):
    """
    Path Constraint: The hammer head must remain aligned with the intended swinging direction.
    The cost is the squared difference between the cosine of the angle and 1 (cos(0) = 1), minus epsilon squared.
    Constraint is satisfied if cost <= 0.
    """
    hammer_vector = keypoints[4] - keypoints[2]  # Hammer orientation
    swing_direction = keypoints[0] - keypoints[4]  # Direction to apple
    hammer_vector_norm = hammer_vector / (np.linalg.norm(hammer_vector) + 1e-8)
    swing_direction_norm = swing_direction / (np.linalg.norm(swing_direction) + 1e-8)
    cos_theta = np.dot(hammer_vector_norm, swing_direction_norm)
    cost = (cos_theta - 1.0)**2 
    return cost


goal_constraints = [stage3_subgoal_constraint1, stage3_subgoal_constraint2]
path_constraints = [stage3_path_constraint1]

In [121]:
def stage3_subgoal_constraint1(eef_pose, keypoints, eef_velocity):
    """
    The hammer head should make contact with the apple at impact.
    The cost is the distance between the hammer head and the apple.
    Constraint is satisfied when cost is close to zero.
    """
    hammer_vector = keypoints[4] - keypoints[2]
    hammer_head_pose = eef_pose + hammer_vector
    cost = np.linalg.norm(hammer_head_pose - keypoints[0])
    return cost



def stage3_subgoal_constraint2(eef_pose, keypoints, eef_velocity):
    """
    The hammer head should hit the apple with desired impact velocity.
    The cost is the difference between desired and actual velocity towards the apple.
    Constraint is satisfied when cost is less than or equal to zero.
    """
    hammer_vector = keypoints[4] - keypoints[2]
    hammer_head_pose = eef_pose + hammer_vector
    # Direction from hammer head to apple
    direction = keypoints[0] - hammer_head_pose
    direction_normalized = direction / np.linalg.norm(direction)
    # Assume hammer head velocity equals eef_velocity
    velocity_towards_apple = np.dot(eef_velocity, direction_normalized)
    v_desired = 1.0  # Desired impact speed in meters per second
    cost = v_desired - velocity_towards_apple  # Constraint satisfied when velocity_towards_apple >= v_desired
    return cost


### Stage 3 Path Constraints



def stage3_path_constraint2(eef_pose, keypoints, eef_velocity):
    """
    Hammer head remains aligned with the intended swing direction during the swing.
    The cost is minimized when the hammer orientation aligns with the swing direction.
    """
    hammer_vector = keypoints[4] - keypoints[2]
    hammer_vector_normalized = hammer_vector / np.linalg.norm(hammer_vector)
    hammer_head_pose = eef_pose + hammer_vector
    # Swing direction vector from hammer head to apple
    swing_direction = keypoints[0] - hammer_head_pose
    swing_direction_normalized = swing_direction / np.linalg.norm(swing_direction)
    # Cost based on alignment
    dot_product = np.dot(hammer_vector_normalized, swing_direction_normalized)
    cost = 1 - dot_product
    return cost


goal_constraints = [stage3_subgoal_constraint1, stage3_subgoal_constraint2]
path_constraints = [stage3_path_constraint1]

In [132]:
aux_args = (og_bounds,
            keypoints_ee,
            keypoint_movable_mask,
            goal_constraints,
            path_constraints,
            ee_pose_homo)

In [133]:
global_opt_result = scipy.optimize.dual_annealing(
                func=objective,
                bounds=bounds,
                args=aux_args,
                maxfun=5000,
                x0=init_sol,
                no_local_search=False,
                minimizer_kwargs={
                    'method': 'SLSQP',
                    'options': {'maxiter': 200}
                },
            )

In [134]:
global_sol = opt_utils.unnormalize_vars(global_opt_result.x, og_bounds)
print(global_sol)

[-0.38165899 -0.05647484  0.82037235 -0.0493054   1.45510888 -0.27478346
  1.75386962  0.08818118  1.56544495]


In [135]:
# Might need to update aux_args
gradient_opt_result = scipy.optimize.minimize(
                fun=objective,
                x0=global_sol,
                args=aux_args,
                bounds=bounds,
                method='SLSQP',
                options={'maxiter': 200},
            )

In [136]:
gradient_sol = opt_utils.unnormalize_vars(gradient_opt_result.x, og_bounds)
print(gradient_sol)

[-0.44376115  0.04683019  0.84673951  2.66612832 -3.12018935  2.54172689
  2.62224217  0.09470423  0.13298365]


In [ ]:
# Final EEF Pose in world frame
final_pose_homo = T.pose2mat([gradient_sol[:3], T.euler2quat(gradient_sol[3:6])])
# Convert movable keypoints from eef frame back to world frame: final keypoints position in world frame
final_keypoints = opt_utils.transform_keypoints(final_pose_homo, keypoints_ee, keypoint_movable_mask)
final_hammer_head = final_keypoints[5]
final_hammer_handle = final_keypoints[3]
final_hammer_vec = final_hammer_head - final_hammer_handle
print(final_hammer_vec)

[ 0.1179431   0.16967617 -0.07594213]
